In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input

In [2]:
pd.set_option('display.max_columns', None) 
df = pd.read_csv('data/pitcher_main_trimmed.csv')
df = df.drop(['player_name', 'game_date'], axis=1)
df

,pitch_type,release_speed,release_pos_x,release_pos_z,pitcher,events,description,zone,p_throws,type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,inning,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,pitch_number,pitch_name,delta_run_exp
0,ff,90.3,1.68,6.60,115399,strikeout,swinging_strike,13.0,l,s,1,2,2008,0.40,0.94,-0.73,1.28,6,-7.021865,-132.301574,-8.362541,6.289000,34.467000,-19.624000,3.57,1.75,4,4-seam fastball,-0.075
1,si,85.0,2.10,6.08,115399,ordinary,ball,14.0,l,b,0,2,2008,1.41,0.17,2.10,1.77,6,-3.316781,-124.853944,-2.798613,14.684000,32.913000,-29.805000,3.57,1.75,3,sinker,0.010
2,si,83.7,1.96,6.36,115399,ordinary,swinging_strike,14.0,l,s,0,1,2008,1.23,0.68,1.31,2.04,6,-4.352248,-122.837225,-3.718380,12.749000,31.332000,-24.795000,3.57,1.75,2,sinker,-0.023
3,ff,91.5,1.68,6.62,115399,ordinary,called_strike,14.0,l,s,0,0,2008,0.70,1.39,0.93,1.80,6,-3.669918,-134.169651,-8.573571,8.963000,34.367000,-14.114000,3.57,1.75,1,4-seam fastball,-0.016
4,ff,91.5,1.52,6.54,115399,field_out,hit_into_play,4.0,l,x,0,2,2008,0.56,1.23,-0.66,2.40,6,-6.935053,-134.094163,-6.479771,8.162000,33.240000,-16.476000,3.38,1.50,3,4-seam fastball,-0.113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9544338,si,96.2,-1.66,5.70,666168,ordinary,foul,4.0,r,s,0,0,2022,-1.79,0.64,-0.41,2.54,9,7.428355,-139.888778,-4.260597,-25.232534,31.096104,-22.963414,3.49,1.60,1,sinker,-0.034
9544339,ff,92.5,-2.49,5.99,608723,ordinary,ball,11.0,r,b,0,0,2022,-0.73,1.46,-0.36,3.99,6,6.965142,-134.626401,-2.524665,-10.246435,26.104120,-13.900579,3.26,1.51,1,4-seam fastball,0.033
9544340,si,91.7,2.12,6.44,571578,ordinary,called_strike,8.0,l,s,0,0,2022,1.16,1.25,-0.02,1.63,1,-7.967493,-133.046151,-8.987455,15.537407,30.338976,-15.552218,3.39,1.55,1,sinker,-0.034
9544341,cu,81.9,-1.63,6.31,641907,ordinary,ball,14.0,r,b,0,0,2022,0.94,-1.16,0.66,1.30,8,3.347026,-119.221540,-2.480205,8.186566,24.508845,-42.874474,3.59,1.53,1,curveball,0.033


In [3]:
X = df.drop('pitcher', axis=1)
y = df['pitcher']

In [4]:
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.15, random_state=42)

In [12]:
X_train_encoded = pd.get_dummies(X_train)
X_train_encoded

,release_speed,release_pos_x,release_pos_z,zone,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,inning,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,pitch_number,delta_run_exp,pitch_type_ab,pitch_type_ch,pitch_type_cs,pitch_type_cu,pitch_type_ep,pitch_type_fa,pitch_type_fc,pitch_type_ff,pitch_type_fo,pitch_type_fs,pitch_type_kc,pitch_type_kn,pitch_type_po,pitch_type_sc,pitch_type_si,pitch_type_sl,pitch_type_st,pitch_type_sv,events_double,events_double_play,events_field_out,events_force_out,events_grounded_into_double_play,events_hit_by_pitch,events_home_run,events_ordinary,events_sac_bunt,events_sac_fly,events_single,events_strikeout,events_triple,events_walk,description_ball,description_blocked_ball,description_bunt_foul_tip,description_called_strike,description_foul,description_foul_bunt,description_foul_pitchout,description_foul_tip,description_hit_by_pitch,description_hit_into_play,description_missed_bunt,description_swinging_strike,description_swinging_strike_blocked,p_throws_l,p_throws_r,type_b,type_s,type_x,pitch_name_4-seam fastball,pitch_name_automatic ball,pitch_name_changeup,pitch_name_curveball,pitch_name_cutter,pitch_name_eephus,pitch_name_forkball,pitch_name_knuckle curve,pitch_name_knuckleball,pitch_name_other,pitch_name_pitch out,pitch_name_screwball,pitch_name_sinker,pitch_name_slider,pitch_name_slow curve,pitch_name_slurve,pitch_name_split-finger,pitch_name_sweeper
1358222,88.6,-2.50,6.50,12.0,0,1,2010,-0.16,1.54,0.77,3.83,6,8.386746,-129.638477,-3.118260,-3.352000,24.993000,-14.693000,2.99,1.34,2,-0.050,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6611309,90.8,2.71,4.91,1.0,2,2,2018,0.16,1.00,-0.60,2.90,8,-8.600608,-132.120820,-1.306391,3.475183,24.780491,-20.364301,3.36,1.56,5,0.000,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1107552,78.9,-2.10,5.64,14.0,1,2,2009,1.37,-0.45,0.83,1.01,5,3.389385,-115.704822,-1.033196,11.262000,21.745000,-35.860000,3.12,1.54,4,0.000,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4769184,76.8,1.56,6.17,12.0,1,0,2015,-0.18,-1.00,1.55,4.11,8,0.357410,-112.636754,5.966510,-1.557000,18.555000,-41.426000,3.26,1.33,2,0.045,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1909293,89.1,1.43,6.43,11.0,2,2,2010,0.48,1.21,-0.07,3.69,3,-4.868222,-130.811846,-2.451734,6.403000,32.702000,-18.387000,3.48,1.82,5,0.000,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8824265,82.4,-1.34,5.95,8.0,1,0,2021,-0.55,0.60,0.23,1.84,4,4.619331,-119.898913,-3.964474,-6.248861,23.561964,-25.798286,3.37,1.53,2,-0.049,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
534506,89.9,-1.22,6.50,9.0,0,0,2008,-1.03,1.77,0.28,1.70,4,6.287704,-131.503253,-9.096979,-12.915000,30.308000,-10.421000,3.62,1.67,1,0.035,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9328706,94.5,-2.76,5.78,9.0,0,0,2022,-0.99,1.36,0.39,1.72,7,10.515625,-136.943173,-7.932851,-14.806080,31.921568,-13.351354,3.10,1.49,1,-0.017,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
933276,92.6,3.00,7.20,7

In [6]:
train_cols = X_train_encoded.columns

In [7]:
X_val_encoded = pd.get_dummies(X_val)
X_val_encoded = X_val_encoded.reindex(columns = train_cols, fill_value = 0)

In [8]:
X_test_encoded = pd.get_dummies(X_test)
X_test_encoded = X_test_encoded.reindex(columns = train_cols, fill_value = 0)

In [9]:
scaler = StandardScaler().fit(X_train_encoded)

In [11]:
X_train_encoded_scaled = scaler.transform(X_train_encoded)
X_val_encoded_scaled = scaler.transform(X_val_encoded)
X_test_encoded_scaled = scaler.transform(X_test_encoded)

array([[ 0.58424614,  1.10143141,  0.63851745, ..., -0.03521646,
        -0.12766017, -0.0681367 ],
       [ 0.91623288, -0.8173168 , -0.62033291, ..., -0.03521646,
        -0.12766017, -0.0681367 ],
       [ 0.08626604, -0.19516138,  0.41871818, ..., -0.03521646,
        -0.12766017, -0.0681367 ],
       ...,
       [-0.74370081, -0.1690204 , -1.91914677, ..., -0.03521646,
        -0.12766017, -0.0681367 ],
       [-0.68836969, -0.80163221,  0.51862694, ..., -0.03521646,
        -0.12766017, -0.0681367 ],
       [ 0.32603424, -0.41474565, -0.26066138, ..., -0.03521646,
        -0.12766017, -0.0681367 ]])

In [19]:
del X_train, X_val, X_test, X_train_encoded, X_val_encoded, X_test_encoded, df, X_temp, y_temp

In [24]:
model = Sequential()
model.add(Dense(100, activation='tanh', input_shape=(X_train_encoded_scaled.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(50, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(1))

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 100)               9100      
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_6 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 51        
Total params: 14,201
Trainable params: 14,201
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.compile(loss='mean_squared_error',
              optimizer=RMSprop(lr=0.001, rho=0.9))

In [28]:
#history = model.fit(X_train_encoded_scaled, y_train.values,
#                    batch_size=32,
 #                   epochs=100,
  #                  verbose=1,
   #                 validation_data=(X_val_encoded_scaled, y_val.values))

Epoch 1/100
228170/228170 [==============================] - 125s 548us/step - loss: 8745481216.0000 - val_loss: 6938869248.0000
Epoch 2/100
228170/228170 [==============================] - 125s 549us/step - loss: 8745482240.0000 - val_loss: 6941497856.0000
Epoch 3/100
228170/228170 [==============================] - 124s 546us/step - loss: 8751605760.0000 - val_loss: 6940276736.0000
Epoch 4/100
228170/228170 [==============================] - 123s 540us/step - loss: 8752733184.0000 - val_loss: 6951265280.0000
Epoch 5/100
228170/228170 [==============================] - 127s 557us/step - loss: 8756203520.0000 - val_loss: 6915631616.0000
Epoch 6/100
228170/228170 [==============================] - 123s 541us/step - loss: 8768276480.0000 - val_loss: 6981296128.0000
Epoch 7/100
228170/228170 [==============================] - 123s 541us/step - loss: 8779408384.0000 - val_loss: 6956211712.0000
Epoch 8/100
228170/228170 [==============================] - 125s 546us/step - loss: 8774138880.0

KeyboardInterrupt: 

In [30]:
y_pred = model.predict(X_val_encoded_scaled)

In [31]:
mae = mean_absolute_error(y_val, y_pred)
print(f"Mean Absolute Error: {mae}")

Mean Absolute Error: 60832.67547426143


In [33]:
r2 = r2_score(y_val, y_pred)
print(f"R-squared: {r2}")

R-squared: 0.497708588737114


In [34]:
y_test_pred = model.predict(X_test_encoded_scaled)

In [35]:
test_mae = mean_absolute_error(y_test, y_test_pred)
print(f"Test Mean Absolute Error: {test_mae}")

Test Mean Absolute Error: 60870.25731059966


In [36]:
test_r2 = r2_score(y_test, y_test_pred)
print(f"Test R-squared: {test_r2}")

Test R-squared: 0.49792488471682417


# Model tuning

In [38]:
model = Sequential()

In [39]:
model.add(Dense(128, activation='relu', input_shape=(X_train_encoded_scaled.shape[1],)))

In [40]:
model.add(Dense(64, activation='relu'))

In [41]:
model.add(Dense(1))

In [42]:
model.compile(optimizer='adam', 
              loss='mean_squared_error')

In [43]:
early_stopping_monitor = EarlyStopping(patience=3)

In [44]:
model.fit(X_train_encoded_scaled, y_train, validation_split=0.2, epochs=50, callbacks=[early_stopping_monitor])

Epoch 1/50
182536/182536 [==============================] - 88s 483us/step - loss: 8890413056.0000 - val_loss: 7356825600.0000
Epoch 2/50
182536/182536 [==============================] - 87s 478us/step - loss: 7330366976.0000 - val_loss: 7322488832.0000
Epoch 3/50
182536/182536 [==============================] - 87s 476us/step - loss: 7303919104.0000 - val_loss: 7312273920.0000
Epoch 4/50
182536/182536 [==============================] - 88s 480us/step - loss: 7284707328.0000 - val_loss: 7286328320.0000
Epoch 5/50
182536/182536 [==============================] - 88s 483us/step - loss: 7177844736.0000 - val_loss: 7132526080.0000
Epoch 6/50
182536/182536 [==============================] - 88s 484us/step - loss: 7090269184.0000 - val_loss: 7073951744.0000
Epoch 7/50
182536/182536 [==============================] - 88s 483us/step - loss: 7013923328.0000 - val_loss: 6992424448.0000
Epoch 8/50
182536/182536 [==============================] - 88s 480us/step - loss: 6935277056.0000 - val_loss: 

KeyboardInterrupt: 

In [45]:
validation_loss = model.evaluate(X_val_encoded_scaled, y_val.values, verbose=0)
print("Validation Loss:", validation_loss)

Validation Loss: 6319580672.0


In [46]:
y_val_pred = model.predict(X_val_encoded_scaled)

In [47]:
r2 = r2_score(y_val.values, y_val_pred)
print("R-squared:", r2)
mae = mean_absolute_error(y_val.values, y_val_pred)
print("Mean Absolute Error:", mae)

R-squared: 0.5450245750459284
Mean Absolute Error: 58432.25783799168


# Deepball inspired RNN

In [16]:
X_train

,pitch_type,release_speed,release_pos_x,release_pos_z,events,description,zone,p_throws,type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,inning,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,pitch_number,pitch_name,delta_run_exp
1358222,ff,88.6,-2.50,6.50,ordinary,foul,12.0,r,s,0,1,2010,-0.16,1.54,0.77,3.83,6,8.386746,-129.638477,-3.118260,-3.352000,24.993000,-14.693000,2.99,1.34,2,4-seam fastball,-0.050
6611309,ff,90.8,2.71,4.91,ordinary,foul,1.0,l,s,2,2,2018,0.16,1.00,-0.60,2.90,8,-8.600608,-132.120820,-1.306391,3.475183,24.780491,-20.364301,3.36,1.56,5,4-seam fastball,0.000
1107552,cu,78.9,-2.10,5.64,ordinary,foul,14.0,r,s,1,2,2009,1.37,-0.45,0.83,1.01,5,3.389385,-115.704822,-1.033196,11.262000,21.745000,-35.860000,3.12,1.54,4,curveball,0.000
4769184,cu,76.8,1.56,6.17,ordinary,ball,12.0,l,b,1,0,2015,-0.18,-1.00,1.55,4.11,8,0.357410,-112.636754,5.966510,-1.557000,18.555000,-41.426000,3.26,1.33,2,curveball,0.045
1909293,ff,89.1,1.43,6.43,ordinary,foul,11.0,l,s,2,2,2010,0.48,1.21,-0.07,3.69,3,-4.868222,-130.811846,-2.451734,6.403000,32.702000,-18.387000,3.48,1.82,5,4-seam fastball,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8824265,ch,82.4,-1.34,5.95,ordinary,swinging_strike,8.0,r,s,1,0,2021,-0.55,0.60,0.23,1.84,4,4.619331,-119.898913,-3.964474,-6.248861,23.561964,-25.798286,3.37,1.53,2,changeup,-0.049
534506,ff,89.9,-1.22,6.50,ordinary,ball,9.0,r,b,0,0,2008,-1.03,1.77,0.28,1.70,4,6.287704,-131.503253,-9.096979,-12.915000,30.308000,-10.421000,3.62,1.67,1,4-seam fastball,0.035
9328706,ff,94.5,-2.76,5.78,ordinary,called_strike,9.0,r,s,0,0,2022,-0.99,1.36,0.39,1.72,7,10.515625,-136.943173,-7.932851,-14.806080,31.921568,-13.351354,3.10,1.49,1,4-seam fastball,-0.017
933276,ff,92.6,3.00,7.20,field_out,hit_into_play,7.0,l,x,1,2,2009,0.90,1.66,-0.68,1.89,2,-11.661832,-135.092120,-10.737449,13.300000,32.713000,-10.212000,3.95,1.66,4,4-seam fastball,-0.160


In [15]:
pitch_name_inputs = Input(shape=(X_train[['pitch_name_4-seam fastball', 'pitch_name_automatic ball', 'pitch_name_changeup', 'pitch_name_curveball', 'pitch_name_cutter', 'pitch_name_eephus', 'pitch_name_forkball', 'pitch_name_knuckle curve', 'pitch_name_knuckleball', 'pitch_name_other', 'pitch_name_pitch out', 'pitch_name_screwball', 'pitch_name_sinker', 'pitch_name_slider', 'pitch_name_slow curve', 'pitch_name_slurve', 'pitch_name_split-finger', 'pitch_name_sweeper']].shape[1],))

KeyError: "None of [Index(['pitch_name_4-seam fastball', 'pitch_name_automatic ball',\n       'pitch_name_changeup', 'pitch_name_curveball', 'pitch_name_cutter',\n       'pitch_name_eephus', 'pitch_name_forkball', 'pitch_name_knuckle curve',\n       'pitch_name_knuckleball', 'pitch_name_other', 'pitch_name_pitch out',\n       'pitch_name_screwball', 'pitch_name_sinker', 'pitch_name_slider',\n       'pitch_name_slow curve', 'pitch_name_slurve', 'pitch_name_split-finger',\n       'pitch_name_sweeper'],\n      dtype='object')] are in the [columns]"

In [ ]:
vxvyvz_inputs = Input(shape=(X_train[['vx0', 'vy0', 'vz0']].shape[1],))